# High Performance Spark Queries with Databricks Delta
Databricks Delta extends Apache Spark to simplify data reliability and boost Spark's performance.

Building robust, high performance data pipelines can be difficult due to: _lack of indexing and statistics_, _data inconsistencies introduced by schema changes_ and _pipeline failures_, _and having to trade off between batch and stream processing_.

With Databricks Delta, data engineers can build reliable and fast data pipelines. Databricks Delta provides many benefits including:
* Faster query execution with indexing, statistics, and auto-caching support
* Data reliability with rich schema validation and rransactional guarantees
* Simplified data pipeline with flexible UPSERT support and unified Structured Streaming + batch processing on a single data source.

### Let's See How Databricks Delta Makes Spark Queries Faster!

In this example, we will see how Databricks Delta can optimize query performance. We create a standard table using Parquet format and run a quick query to observe its latency. We then run a second query over the Databricks Delta version of the same table to see the performance difference between standard tables versus Databricks Delta tables. 

Simply follow these 4 steps below:
* __Step 1__ : Create a standard Parquet based table using data from US based flights schedule data
* __Step 2__ : Run a query to to calculate number of flights per month, per originating airport over a year
* __Step 3__ : Create the flights table using Databricks Delta and optimize the table.
* __Step 4__ : Rerun the query in Step 2 and observe the latency. 

__Note:__ _Throughout the example we will be building few tables with a 10s of million rows. Some of the operations may take a few minutes depending on your cluster configuration._

In [2]:
%fs rm -r /tmp/flights_parquet 

res0: Boolean = true

In [3]:
%fs rm -r /tmp/flights_delta

res1: Boolean = true

In [4]:
flights = spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .load("/databricks-datasets/asa/airlines/2008.csv")

In [5]:
flights.write.format("parquet").mode("overwrite").partitionBy("Origin").save("/tmp/flights_parquet")

Once step 1 completes, the "flights" table contains details of US flights for a year. 

Next in Step 2, we run a query that get top 20 cities with highest monthly total flights on first day of week.

In [7]:
from pyspark.sql.functions import count

flights_parquet = spark.read.format("parquet").load("/tmp/flights_parquet")

display(flights_parquet.filter("DayOfWeek = 1").groupBy("Month","Origin").agg(count("*").alias("TotalFlights")).orderBy("TotalFlights", ascending=False).limit(20))

Month,Origin,TotalFlights
6,ATL,6046
3,ATL,6019
12,ATL,5800
9,ATL,5722
6,ORD,5241
3,ORD,5072
9,ORD,4931
7,ATL,4894
8,ATL,4821
4,ATL,4798


Once step 2 completes, you can observe the latency with the standard "flights_parquet" table. 

In step 3 and step 4, we do the same with a Databricks Delta table. This time, before running the query, we run the `OPTIMIZE` command with `ZORDER` to ensure data is optimized for faster retrieval.

In [9]:
flights.write.format("delta").mode("overwrite").partitionBy("Origin").save("/tmp/flights_delta")

In [10]:
display(spark.sql("DROP TABLE  IF EXISTS flights"))

display(spark.sql("CREATE TABLE flights USING DELTA LOCATION '/tmp/flights_delta'"))
                  
display(spark.sql("OPTIMIZE flights ZORDER BY (DayofWeek)"))

path
null


In [11]:
flights_delta = spark.read.format("delta").load("/tmp/flights_delta")

display(flights_delta.filter("DayOfWeek = 1").groupBy("Month","Origin").agg(count("*").alias("TotalFlights")).orderBy("TotalFlights", ascending=False).limit(20))

Month,Origin,TotalFlights
6,ATL,6046
3,ATL,6019
12,ATL,5800
9,ATL,5722
6,ORD,5241
3,ORD,5072
9,ORD,4931
7,ATL,4894
8,ATL,4821
4,ATL,4798


The query over the Databricks Delta table runs much faster after `OPTIMIZE` is run. How much faster the query runs can depend on the configuration of the cluster you are running on, however should be **5-10X** faster compared to the standard table.